# L4: Part-of-speech tagging

Part-of-speech tagging is the task of labelling the words (tokens) of a sentence with parts-of-speech such as noun, adjective, and verb. In this lab you will implement the simple, autoregressive fixed-window tagger that was presented in Lecture&nbsp;4.2.

## The data set

The data set for the lab is the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org), a corpus containing more than 16,000 sentences (254,000&nbsp;tokens) annotated with, among other things, parts-of-speech. The Universal Dependencies Project distributes its data in the [CoNLL-U format](https://universaldependencies.org/format.html), but for this lab we have converted the data into a simpler format: words and their part-of-speech tags are separated by tabs, sentences are separated by empty lines. The code in the next cell defines a container class for data with this format.

In [ ]:
class Dataset():

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        tmp = []
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            for line in lines:
                line = line.rstrip()
                if line:
                    tmp.append(tuple(line.split('\t')))
                else:
                    yield tmp
                    tmp = []

We load the training data and the development data for this lab:

In [ ]:
train_data = Dataset('train.txt')
dev_data = Dataset('dev.txt')


Both data sets consist of **tagged sentences**. On the Python side of things, a tagged sentence is represented as a list of string pairs, where the first component of each pair represents a word token and the second component represents the word’s tag. The possible tags are listed and exemplified in the [Annotation Guidelines](http://universaldependencies.org/u/pos/all.html) of the Universal Dependencies Project. Run the next code cell to see an example of a tagged sentence.

In [ ]:
list(train_data)[42]

[('There', 'PRON'),
 ('has', 'AUX'),
 ('been', 'VERB'),
 ('talk', 'NOUN'),
 ('that', 'SCONJ'),
 ('the', 'DET'),
 ('night', 'NOUN'),
 ('curfew', 'NOUN'),
 ('might', 'AUX'),
 ('be', 'AUX'),
 ('implemented', 'VERB'),
 ('again', 'ADV'),
 ('.', 'PUNCT')]

## Tagger interface

The tagger that you will implement in this lab follows a simple interface with just one method:

In [ ]:
class Tagger(object):

    def predict(self, sentence):
        raise NotImplementedError

The single method of this interface has the following specification:

**predict** (*self*, *sentence*)

> Returns the list of predicted tags (a list of strings) for a single *sentence* (a list of string tokens).

One trivial implementation of this interface is a tagger that always predicts the same tag for every word, independently of the input:

In [ ]:
class ConstantTagger(Tagger):

    def __init__(self, the_tag):
        self.the_tag = the_tag

    def predict(self, words):
        return [self.the_tag] * len(words)

## Problem 1: Implement an evaluation function

Your first task is to implement a function that computes the accuracy of a tagger on gold-standard data.

In [ ]:
def accuracy(tagger, gold_data):
    # TODO: Replace the next line with your own code
    correct = 0
    total = 0
    for gold in gold_data:
      words = [j[0] for j in gold]
      predict = tagger.predict(words)
      for j in range(len(predict)):
        if predict[j] == gold[j][1]:
          correct +=1
        total += 1
    return correct/total

Your implementation should conform to the following specification:

**accuracy** (*tagger*, *gold_data*)

> Computes the accuracy of the *tagger* on the gold-standard data *gold_data* (an iterable of tagged sentences) and returns it as a float. Recall that the accuracy is defined as the percentage of tokens to which the tagger assigns the correct tag (as per the gold standard).

### 🤞 Test your code

Test your code by computing the accuracy on the development set of a trivial tagger that tags each word as a noun. The expected value is 16.69%.

In [ ]:

tagger = ConstantTagger('NOUN')

accuracy(tagger, dev_data)




0.1668919993637665

## Problem 2: Implement a baseline

Before you start working on the tagger as such, we ask you to first implement a simple baseline:

> Tag each input word with the most frequent tag for that word in the training data. If an input word does not occur in the training data, tag it with the overall most frequent tag in the training data. Break ties by choosing that tag which comes first in the alphabetical order.

To implement the baseline, you need to implement both a class `BaselineTagger` and a function `train_baseline`. A `BaselineTagger` has two fields: a dictionary mapping each word in the training data to the most frequent tag for that word, and a string representing the fallback tag (overall most frequent tag in the training data). Both of these fields are set in the `train_baseline` function.

In [ ]:
class BaselineTagger(Tagger):
    def __init__(self):
        self.most_frequent = {}
        self.fallback = None

    def predict(self, words):
        # TODO: Replace the next line with your own code
        predict = []
        for word in words:
          if word in self.most_frequent:
            predict.append(self.most_frequent[word])
          else:
            predict.append(self.fallback)
        return predict

def train_baseline(train_data):
    # TODO: Replace the next line with your own code
    frequency = {}
    most_common = {}
    for sublist in train_data:
      for word in sublist:
        if word[0] in frequency:
          if word[1] in most_common:
            most_common[word[1]] += 1
          else:
            most_common[word[1]] = 1
          if word[1] in frequency[word[0]]:
            frequency[word[0]][word[1]] += 1
          else:
            frequency[word[0]][word[1]] = 1
        else:
          frequency[word[0]] = {word[1] : 1}
    tagger = BaselineTagger()
    tagger.fallback = max(most_common, key=most_common.get)
    for word in frequency:
      # Sort dictionary so that it chooses on alphabetic order if several tags has the same frequency
      frequency[word] = dict(sorted(frequency[word].items()))
      frequency[word] = max(frequency[word], key=frequency[word].get)
    tagger.most_frequent = frequency

    return tagger

### 🤞 Test your code

Test your implementation by computing the accuracy of the baseline tagger on the development data. The expected value is 85.61%.

In [ ]:
tagger = train_baseline(list(train_data))

accuracy(tagger, dev_data)


0.8560521711468109

## Problem 3: Create the vocabularies

As in previous labs, you will need an explicit representation of your vocabulary. Here we actually have two vocabularies: one for the words and one for the tags. Both should be represented as dictionaries that map words/tags to a contiguous range of integers, starting at zero.

The next cell contains skeleton code for a function `make_vocabs` that constructs the two vocabularies from gold-standard data. The code cell also defines a name for the ‘unknown word’ (`UNK`) and for an additional pseudoword that you will use as a placeholder for undefined values (`PAD`).

In [ ]:
PAD = '<pad>'
UNK = '<unk>'

def make_vocabs(gold_data):
    # TODO: Replace the next line with your own code
    counter = []
    vocab_words = {}
    vocab_tags = {}
    for data in gold_data:
      for word in data:
        if word[0] not in vocab_words:
          vocab_words[word[0]] = len(vocab_words)
        if word[1] not in vocab_tags:
          vocab_tags[word[1]] = len(vocab_tags)
    vocab_words['<pad>'] = len(vocab_words)
    vocab_tags['<pad>'] = len(vocab_tags)
    vocab_words['<unk>'] = len(vocab_words)

    #Make reverse vocab_tags
    vocab_tags_reverse = {}
    for key, value in vocab_tags.items():
      vocab_tags_reverse[value] = key


    return vocab_words, vocab_tags, vocab_tags_reverse

Complete the code according to the following specification:

**make_vocabs** (*gold_data*)

> Returns a pair of dictionaries mapping the unique words and tags in the gold-standard data *gold_data* (an iterable over tagged sentences) to contiguous ranges of integers starting at zero. The word dictionary contains the pseudowords `PAD` (index&nbsp;0) and `UNK` (index&nbsp;1); the tag dictionary contains `PAD` (index&nbsp;0).

### 🤞 Test your code

Test your implementation by computing the total number of unique words and tags in the training data (including the pseudowords). The expected values are 19,674&nbsp;words and 18&nbsp;tags.

In [ ]:
vocab_words, vocab_tags, vocab_tags_reverse = make_vocabs(train_data)
print(len(vocab_words), len(vocab_tags))

19674 18


## Problem 4: Fixed-window tagger

Your main task in this lab is to implement a complete, autoregressive part-of-speech tagger based on the fixed-window architecture. This implementation has four parts: the fixed-window model; a tagger that uses the fixed-window model to make predictions; a function that generates training examples for the tagger; and the training function.

**⚠️ We expect that solving this problem will take you the longest time in this lab.**

### Problem 4.1: Implement the fixed-window model

The architecture of the fixed-window model is presented in Lecture&nbsp;4.2. An input to the network takes the form of a $k$-dimensional vector of word ids and/or tag ids. Each integer $i$ is mapped to an $e_i$-dimensional embedding vector. These vectors are concatenated to form a vector of length $e_1 + \cdots + e_k$, and sent through a feed-forward network with a single hidden layer and a rectified linear unit (ReLU).

#### Default features

We ask you to implement a fixed-window model with the following features ($k=4$):

0. current word
1. previous word
2. next word
3. tag predicted for the previous word

Whenever the value of a feature is undefined, you should use the special value `PAD`.

#### Embedding specifications

To make your implementation of the fixed-window model useful for a range of different applications (including the parser that you will build in lab&nbsp;5), it should support other feature sets than the default model. To this end, the constructor of your model should accept a list of what we call *embedding specifications*. An embedding specification is a triple $(m, n, e)$ consisting of three integers. Such a triple specifies that the model should include $m$ instances of an embedding from $n$ items to vectors of size $e$. All of the $m$ instances are to share their weights. In this lab, the embeddings will be embeddings for words and tags. For example, to instantiate the default feature model, you would initialise the model with the following specifications:

``
[(3, num_words, word_dim), (1, num_tags, tag_dim)]
``

This specifies that the model should use 3 instances of an embedding from *num_words* words to vectors of length *word_dim*, and 1 instance of an embedding from *num_tags* tags to vectors of length *tag_dim*. All 3 instances of the word embedding would share their weights. If you rather wanted to have word embeddings with separate weights, you would initialise the model with the following specifications:

``
[(1, num_words, word_dim), (1, num_words, word_dim), (1, num_words, word_dim), (1, num_tags, tag_dim)]
``

We recommend that you initialize the weights of each embedding with values drawn from $\mathcal{N}(0, 10^{-2})$.

#### Hyperparameters

The network architecture introduces a number of hyperparameters. The following choices are reasonable defaults:

* width of each word embedding: 50
* width of each tag embedding: 10
* size of the hidden layer: 100

The next cell contains skeleton code for the implementation of the fixed-window model.

In [ ]:
import torch.nn as nn
import torch

class FixedWindowModel(nn.Module):
    def __init__(self, embedding_specs, hidden_dim = 100, output_dim=100):
        super().__init__()
        moduleList = nn.ModuleList([])
        self.embedding_layers = nn.ModuleList([])
        self.linear_input = 0
        n_words = 0
        n_tags = 0
        self.k = 0
        index = 1
        for specs in embedding_specs:
          if index == 0:
            embedding = nn.Embedding(specs[1], specs[2])
            embedding.weight = nn.parameter.Parameter(glove)
            index += 1
          else:
              embedding = nn.Embedding(specs[1], specs[2])
              embedding.weight.data.normal_(0.0,0.01)
          self.embedding_layers.append(embedding)
          self.linear_input += specs[0]*specs[2]
          self.k += specs[0]
          if specs[1] > n_words:
            n_words = specs[1]
          if specs[1] != n_words:
            n_tags = specs[1]
        self.instances = [value[0] for value in embedding_specs]
        self.hidden_linear = nn.Linear(self.linear_input, hidden_dim, bias = True)
        self.relu = torch.relu
        self.linear = nn.Linear(hidden_dim, n_tags, bias = True)

    def forward(self, features):
        index = 0
        embedd = []
        for i in range(len(self.embedding_layers)):
          embedd.append(self.embedding_layers[i](features[:,index:index+self.instances[i]].long()))
          index += self.instances[i]
          if self.instances[i] > 1 :
            embedd[i] = embedd[i].view(len(features),1,-1)
        concat = torch.cat(tuple(embedd), dim=2)
        hidden = self.hidden_linear(concat)
        relu = self.relu(hidden)
        output = self.linear(relu)
        return output

Your implementation should meet the following specification:

**__init__** (*self*, *embedding_specs*, *hidden_dim*, *output_dim*)

> A fixed-window model is initialized with a list of specifications for the embeddings the network should use (*embedding_specs*), the size of the hidden layer (*hidden_dim*), and the size of the output layer (*output_dim*).

**forward** (*self*, *features*)

> Computes the network output for a given feature representation *features*. This is a tensor of shape $B \times k$ where $B$ is the batch size (number of samples in the batch) and $k$ is the total number of embeddings specified upon initialisation. For example, for the default feature model, $k=4$, as this model includes 3 (weight-sharing) word embeddings and 1 tag embedding.

#### 💡 Hint on the implementation

You will have to construct embeddings based on the embedding specifications. It is natural to store these embeddings in a list- or dictionary-valued attribute of the `FixedWindowModel` object. However, in order to expose the embeddings to the auto-differentiation magic of PyTorch (so that their weights are updated during training), you must instead store them in an [`nn.ModuleList`](https://pytorch.org/docs/stable/nn.html#torch.nn.ModuleList) or [`nn.ModuleDict`](https://pytorch.org/docs/stable/nn.html#torch.nn.ModuleDict).

In [ ]:
embedding_specs = [(3, len(vocab_words), 50), (1, len(vocab_tags), 10)]
model = FixedWindowModel(embedding_specs)

### Problem 4.2: Implement the tagger

The next step is to implement the tagger itself. The tagger will use the simple algorithm that was presented in Lecture&nbsp;4.2: It processes an input sentence from left to right, and at each position, predicts the tag for the current word based on the features extracted from the current feature window.

In [ ]:
class FixedWindowTagger(Tagger):

    def __init__(self, vocab_words, vocab_tags, word_dim=50, tag_dim=10, hidden_dim=100):
        self.embedding_specs = [(3, len(vocab_words), word_dim), (1, len(vocab_tags), tag_dim)]
        self.model = FixedWindowModel(embedding_specs, len(vocab_tags), hidden_dim)
        self.vocab_words = vocab_words
        self.vocab_tags = vocab_tags

    def featurize(self, words, i, pred_tags):
        output = torch.zeros(4)
        output[0] = words[i]
        if i == 0:
          output[1] = self.vocab_words['<pad>']
          output[3] = self.vocab_tags['<pad>']
          if len(words)<2:
            output[2] = self.vocab_words['<pad>']
          else:
            output[2] = words[i+1]
        elif len(words) == (i+1):
          output[1] = words[i-1]
          output[3] = pred_tags[i-1]
          output[2] = self.vocab_words['<pad>']
        else:
          output[1] = words[i-1]
          output[3] = pred_tags[i-1]
          output[2] = words[i+1]
        return output

    def predict(self, words):
        pred_tags = torch.zeros(len(words))
        words_id = torch.zeros(len(words))
        for i in range(len(words)):
          if words[i] in self.vocab_words:
            words_id[i] = self.vocab_words[words[i]]
          else:
            words_id[i] = self.vocab_words['<unk>']
        for i in range(len(words_id)):
          features = self.featurize(words_id, i, pred_tags)
          features = features.unsqueeze(0)
          tag = self.model.forward(features)
          index_tag = tag.argmax()
          pred_tags[i] = index_tag.item()
        pred_tags_list = []
        for i in range(len(pred_tags)):
          pred_tags_list.append(vocab_tags_reverse[int(pred_tags[i].item())])

        return pred_tags_list

Complete the skeleton code by implementing the methods of this interface:

**__init__** (*self*, *vocab_words*, *vocab_tags*, *word_dim* = 50, *tag_dim* = 10, *hidden_dim* = 100)

> Creates a new fixed-window model of appropriate dimensions and sets up any other data structures that you consider relevant. The parameters *vocab_words* and *vocab_tags* are the word vocabulary and tag vocabulary. The parameters *word_dim* and *tag_dim* specify the embedding width for the word embeddings and tag embeddings.

**featurize** (*self*, *words*, *i*, *pred_tags*)

> Extracts features from the specified tagger configuration according to the default feature model. The configuration is specified in terms of the words in the input sentence (*words*, a list of word ids), the position of the current word (*i*), and the list of already predicted tags (*pred_tags*, a list of tag ids). Returns a tensor that can be fed to the fixed-window model.

**predict** (*self*, *words*)

> Processes the input sentence *words* (a list of string tokens) and makes calls to the fixed-window model to predict the tag of each word. Returns the list of the predicted tags (strings).

### Problem 4.3: Generate the training examples

Your next task is to implement a function that generates the training examples for the tagger. You will train the tagger as usual, using minibatch training.

In [ ]:
def training_examples(vocab_words, vocab_tags, gold_data, tagger, batch_size=100):
    batch = torch.zeros((batch_size,4))
    tags = torch.zeros(batch_size)
    index = 0
    for data in gold_data:
      words_id = torch.zeros(len(data))
      tags_id = torch.zeros(len(data))
      for i, word in enumerate(data):
        words_id[i] = vocab_words[word[0]]
        tags_id[i] = vocab_tags[word[1]]
      for i in range(len(words_id)):
        row = tagger.featurize(words_id, i, tags_id)
        tags[index] = tags_id[i]
        batch[index] = row
        if index == (batch_size - 1):
          yield batch, tags
          index = -1
          batch = torch.zeros((batch_size,4))
          tags = torch.zeros(batch_size)
        index += 1
    if index < batch_size:
      batch = batch[0:index]
      tags = tags[0:index]
      yield batch, tags

tagger = FixedWindowTagger(vocab_words, vocab_tags)
for batch, tags in training_examples(vocab_words, vocab_tags, train_data, tagger):

    break

Your code should comply with the following specification:

**training_examples** (*vocab_words*, *vocab_tags*, *gold_data*, *tagger*, *batch_size* = 100)

> Iterates through the given *gold_data* (an iterable of tagged sentences), encodes it into word ids and tag ids using the specified vocabularies *vocab_words* and *vocab_tags*, and then yields batches of training examples for gradient-based training. Each batch contains *batch_size* examples, except for the last batch, which may contain fewer examples. Each example in the batch is created by a call to the `featurize` function of the *tagger*.

### Problem 4.4: Training loop

What remains to be done is the implementation of the training loop. This should be a straightforward generalization of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=1, batch_size=100, lr=1e-2):
    # Initialize the model and tagger
    tagger = FixedWindowTagger(vocab_words, vocab_tags)

    # Initialize the optimizer. Here we use Adam rather than plain SGD
    optimizer = optim.Adam(tagger.model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        tagger.model.train()
        for batch, tags in training_examples(vocab_words, vocab_tags, train_data, tagger):
            optimizer.zero_grad()
            output = tagger.model.forward(batch)
            output = output.squeeze(1)
            tags = tags.long()
            loss = F.cross_entropy(output, tags)
            loss.backward()
            optimizer.step()
    return tagger

Here is the specification of the training function:

**train_fixed_window** (*train_data*, *n_epochs* = 1, *batch_size* = 100, *lr* = 1e-2)

> Trains a fixed-window tagger from a set of training data *train_data* (an iterable over tagged sentences) using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*.

The next code cell trains a tagger and evaluates it on the development data:

In [ ]:
tagger = train_fixed_window(train_data, n_epochs=2)
print('{:.4f}'.format(accuracy(tagger, dev_data)))

0.8823


In [ ]:
class Dataset():

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            tmp = []
            for line in lines:
                if not line.startswith('#'):  # Skip lines with comments
                    line = line.rstrip()
                    if line:
                        columns = line.split('\t')
                        if columns[0].isdigit():  # Skip range tokens
                            tmp.append(columns)
                    else:
                        yield tmp
                        tmp = []

with open('en_ewt-ud-dev-retagged.conllu', 'wt') as target:
    for sentence in Dataset('en_ewt-ud-dev.conllu'):
        words = [columns[1] for columns in sentence]
        for i, t in enumerate(tagger.predict(words)):
            sentence[i][3] = t
        for columns in sentence:
            print('\t'.join(c for c in columns), file=target)
        print(file=target)

# (columns[1], columns[3], int(columns[6])))

In [ ]:
glove = torch.load('glove.pt')
print(glove.shape)

torch.Size([19674, 50])


**⚠️ Your submitted notebook must contain output demonstrating at least 88% accuracy on the development set.**

## Problem 5: Pre-trained embeddings (reflection)

Many neural systems for natural language processing use pre-trained word embeddings, either to augment or to replace randomly initialised task-based embeddings. In this problem, you will investigate whether pre-trained embeddings help your part-of-speech tagger.

The file `glove.pt` contains a PyTorch tensor containing 50-dimensional pre-trained word embeddings from the [GloVe project](https://nlp.stanford.edu/projects/glove/). You can load this tensor using the command

```
glove = torch.load('glove.pt')
```

and should be able to use it as a drop-in replacement for your randomly initialized word embeddings, assuming that the words in your vocabulary are numbered in the order in which they are found in the training data. Have a look at the documentation of the class [`nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) to learn how to do this.

Run experiments to assess the effect that pre-trained embeddings have on (a)&nbsp;the accuracy of the tagger, and (b)&nbsp;the speed of learning, i.e., the number of training examples it takes to reach a certain loss. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* How did you integrate the pre-trained embeddings into your system? What did you measure? What results did you get?
* Based on what you know about word embeddings and transfer learning, did you expect your results? How do you explain them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

When we intialized our embeddings we used the glove instance instead of those from embedding_specs. We used the glove as a parameter for the first embedding but not on the embedding of the tags since that embedding had different shape. We meassured the speed of learning and the accuracy. From this we could see that the accuracy was pretty much the same using pretrained embeddings as for not pretrained embeddings, for two epochs we got an accuracy of 89 % when not using pretrained embeddings and a accuracy of 89 % when using pretrained embeddings. For the speed of learning the pretrained embeddings went from a loss of around 3 to a loss of around 0.08 over two epochs and this was the same for the not pretrained embeddings. We did expect that pretrained embeddings would be better from the start having a lower loss rate than it had, but that it would learn slower than the not pretrained embeddings. In this lab we learned how to use different embeddings when embedding several inputs using nn.ModuleList().

**🥳 Congratulations on finishing this lab! 🥳**